In [2]:
import torch
import ray
from ray import tune
from ray.tune.logger import DEFAULT_LOGGERS

import json
import sys
from pathlib import Path

# TODO Needed to install tensorboard separately
from ghrp.model_definitions.def_simclr_ae_module import SimCLRAEModule
from ghrp.checkpoints_to_datasets.dataset_simclr import SimCLRDataset

# set which hyper-representation to load

PATH_ROOT = Path("./../data/hyper_representations/mnist")
# PATH_ROOT = Path("./../data/hyper_representations/svhn")
# PATH_ROOT = Path("./../data/hyper_representations/cifar10")
# PATH_ROOT = Path("./../data/hyper_representations/stl10")

# load config
config_path = PATH_ROOT.joinpath('config_ae.json')
config = json.load(config_path.open('r'))
config['dataset::dump'] = PATH_ROOT.joinpath('dataset.pt').absolute()

# set resources
gpus = 1 if torch.cuda.is_available() else 0
cpus = 4
resources_per_trial = {"cpu": cpus, "gpu": gpus}

device = torch.device('cuda') if gpus>0 else torch.device('cpu')
config['device'] = device

# Instanciate model
module = SimCLRAEModule(config)
print(config)

from weight_diffusion.data.modelzoo_with_latent_dataset import ModelZooWithLatentDataset

from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('tbs17/MathBERT', output_hidden_states=True)

# ds = ModelZooWithLatentDataset(
#     data_dir=Path("./../data/tune_zoo_mnist_uniform/"),
#     checkpoint_property_of_interest="validation_loss",
#     openai_coefficient=4.185,
#     split="train",
#     encoder=module,
#     device=device,
#     tokenizer=tokenizer
# )


init attn encoder
## encoder -- use index_dict
model: use only positive contrast loss
initialze projection head
{'model::N_attention_blocks': 2, 'model::N_attention_heads': 12, 'model::attention_hidden_dim': 1380, 'model::bottleneck': 'linear_bounded', 'model::compression': 'token', 'model::decompression': 'linear', 'model::decoder_bias': False, 'model::dim_attention_embedding': 1428, 'model::dropout': 0.1, 'model::encoding': 'neuron', 'model::i_dim': 2464, 'model::index_dict': {'channels_in': [1, 8, 6, 4, 20], 'idx_length': [208, 1206, 100, 740, 210], 'idx_start': [0, 208, 1414, 1514, 2254], 'kernel_no': [8, 6, 4, 20, 10], 'kernel_size': [25, 25, 4, 9, 1], 'layer': [[0, 'conv2d'], [3, 'conv2d'], [6, 'conv2d'], [9, 'fc'], [11, 'fc']]}, 'model::latent_dim': 700, 'model::normalize': True, 'model::projection_head_batchnorm': False, 'model::projection_head_hdim': 400, 'model::projection_head_layers': 4, 'model::projection_head_odim': 50, 'model::type': 'transformer', 'optim::lr': 0.0001, '

In [3]:
from weight_diffusion.data.gpt_dataset import GptDataset

ds = GptDataset(
    data_dir=Path("./../data/tune_zoo_mnist_uniform/"),
    checkpoint_property_of_interest="validation_loss",
    openai_coefficient=4.185,
    split="train"
)

Loading Models:   0%|          | 0/10 [00:00<?, ?it/s]

compute 2 random permutations for layer 0 - 0
compute 2 random permutations for layer 1 - 3
compute 2 random permutations for layer 2 - 6
compute 2 random permutations for layer 3 - 9
prepared 10 permutations
prepare permutation dicts


Indexing G.pt dataset: 100%|██████████| 10/10 [00:00<00:00, 20.05it/s]


In [4]:
ds[0]

{'parameters_0': tensor([2.0695, 3.4705, 2.9288,  ..., 0.0418, 0.0418, 0.0418]),
 'parameters_1': tensor([ 1.6452,  3.7763,  3.5833,  ...,  0.1452, -0.2637,  0.4922]),
 'validation_loss_0': 2.021651029586792,
 'validation_loss_1': 1.9351145029067993}

In [1]:
checkpoint = ds.data_sample
f = open('./../data/tune_zoo_mnist_uniform/config.json')
config = json.load(f)

from ghrp.model_definitions.def_net import NNmodule

base_model1 = NNmodule(config, verbosity=0)
base_model2 = NNmodule(config, verbosity=0)


import torch
import torchvision
import torchvision.transforms as transforms
batch_size = 1

transform = transforms.Compose([
    # you can add other transformations in this list
    transforms.ToTensor()
])

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False)

permutations = ds[0]


output = []
temp = []
perm1 = permutations[0]
perm2 = permutations[1]
print(perm1["module_list.0.weight"])
print(perm2["module_list.0.weight"])
base_model1.model.load_state_dict(perm1)
base_model2.model.load_state_dict(perm2)

torch.set_printoptions(precision=10)

with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        output1 = base_model1(images)
        output2 = base_model2(images)
        print(output1[0]- output2[0])
        print(output1[0])
        print(output2[0])
        print(output1[0][0])
        print(output2[0][0])
        _, predicted1 = torch.max(output1.data, 1)
        _, predicted2 = torch.max(output2.data, 1)
        print(predicted1 == predicted2)
        break



# TODO Test if flattened checkpoint is same using the two methods

NameError: name 'ds' is not defined

In [2]:
# load checkpoint
checkpoint_path = PATH_ROOT.joinpath('checkpoint_ae.pt')
checkpoint = torch.load(checkpoint_path,map_location=device)

# load checkpoint to model
module.model.load_state_dict(checkpoint)

# load dataset
dataset_path = PATH_ROOT.joinpath('dataset.pt')
dataset = torch.load(dataset_path)

# get test weights
weights_test = dataset['testset'].__get_weights__()
print(weights_test.shape)
print(weights_test)
print(weights_test[754])

# forward propagate test weights
with torch.no_grad():
    z, y = module.forward(weights_test.to(device))

# z are the latent representations, y the reconstructed weights
print(z.shape)
print(y.shape)

torch.Size([755, 2464])
tensor([[ 0.1057,  0.5063,  0.7677,  ...,  0.1089, -0.2859, -0.0415],
        [ 0.1394,  0.5325,  0.7727,  ...,  0.1310, -0.3042, -0.0491],
        [ 0.1374,  0.5240,  0.7612,  ...,  0.1481, -0.3287, -0.0630],
        ...,
        [-0.6401, -0.2447,  0.1664,  ..., -0.0060, -0.1384, -0.1888],
        [-0.6525, -0.2684,  0.1314,  ...,  0.0011, -0.1421, -0.2065],
        [-0.6510, -0.2968,  0.0874,  ...,  0.0124, -0.1574, -0.2247]])
tensor([-0.6510, -0.2968,  0.0874,  ...,  0.0124, -0.1574, -0.2247])
torch.Size([755, 700])
torch.Size([755, 2464])


In [3]:
print(y[0].shape)
print(y[1].shape)
print([y[0], y[1]])

ls = y[0].tolist()
print(ls)
test = torch.tensor([ls])
print(test)

# TODO Check if this operation loses floating point info

torch.Size([2464])
torch.Size([2464])
[tensor([-0.1885,  0.1287,  0.4737,  ...,  0.0332, -0.0883, -0.0474]), tensor([-0.1688,  0.1368,  0.4810,  ...,  0.0360, -0.0885, -0.0518])]
[-0.18846958875656128, 0.1287471204996109, 0.4737478196620941, -0.04170406609773636, -0.6226829290390015, 0.45010483264923096, 1.2612406015396118, 0.8412647247314453, 0.18783283233642578, -0.5810459852218628, 0.2705056667327881, 0.7415896654129028, 0.08787162601947784, 0.25182032585144043, -0.20636172592639923, -0.18356674909591675, 0.08819474279880524, 0.06885562092065811, 0.15804779529571533, -0.6367818713188171, -0.6543688178062439, -0.31292828917503357, -0.11141198873519897, -0.07415363192558289, -0.6733338832855225, -0.3693573772907257, -0.14116783440113068, -0.15709049999713898, 0.1236235499382019, 0.11098845303058624, 0.12124721705913544, 0.05510734021663666, 0.1836421638727188, 0.49008700251579285, 0.5012267231941223, 0.4149497151374817, 1.1869255304336548, 1.3410305976867676, 0.8311110734939575, 0.577

Loading Models:   0%|          | 0/10 [00:00<?, ?it/s]


AttributeError: 'GptDataset' object has no attribute 'permute_layers'